<h1><font color="blue">Tropical Restaurant in Toronto</font></h1>

This project is about evaluate the strengths and weaknesses of opening a tropical restaurant in Toronto.

### Table of contents

- [Introduction / Business Understanding](#introduction)
- [Data](#data)
- [Methodology](#methodology)
- [Analysis](#analysis)
- [Conclusion](#conclusion)


## Introduction / Business Understanding<a name="introduction"></a>

### Situation description

Noor is young African chef that is very passionate about cooking and gastronomy. After an holiday trip, 3 years ago, to visit a friend, he fell in love with Toronto and decided to settle over there and open a restaurant.

Noor is interested in gastronomy from all over the world and enforced that interest by many internships in China, Mongolia, Thailand, Cambodge, Brazil, Bolivie, Peru and in many countries in Africa like Ivory Coast, Benin, Togo, Cameroon, Senegal, Bissau, Namibia, Ethiopia and Mozambique. His gastronomy's richness is for him, an element that will open his restaurant a path to success.

After having closed each of the steps of his project, from gathering the funds to the obtention of the necessary permits, now comes the time where he has to decide where his restaurant will be settled.


### Problem presentation 


In order to find the right place for his restaurant, Noor needs to be familiar with the city and its surroundings and has hints on the affordable trendings places available. As a newcomer, he's obviously unable to have these informations.

However, despite not knowing very well the city, he knows the kinds of neighborhoods he's looking for in order to maximize his chances of making a positive impact of his customer base. Here are his criteria (in disorder) :
<font color="blue">
- a venue in a cosmopilite surroundinds, where you can see people from several origins,
- a venue not in a residential neighborhood, but close to a business place, to make it possible for customers to come lunch or order and be delivered in competitive durations,
- a venue in a safe neighborhood,
- a venue in a not too expensive borough,
- a place where there can have safe night activities, especially in weekends. 
</font>

The important question here is : what would be the adequate place for Noor to settle his restaurant and make an great first impression on his customer base?


## Data<a name="data"></a>

To solve Noor's problem, we will need to gather Toronto location data and put them in competition in order to fit the criteria he expoeses in the best way possible. 

__Foursquare__ appears to be an adequate tool to help in the resolution of our problem. In addition to this data, we will also need to have :
- dataset of Toronto residential neighborhoods and their locations, if possible.
- dataset of commercial housing prices in Toronto, if possible dispatched by neighborhoods.
- dataset of crimes rates by neighborhoods in Toronto, if exists.



First of all, we will study Downtown Toronto (DT). By using the ratings of the trending venues in DT, we will have a view of the number and the kind of people that appreciate thoses places. We will thus be able to answer the first criterion.

We will then confront these places with the dataset of Toronto residential neighborhoods to help eliminate those that are in residential areas.

Bu using the dataset of crimes rates by neighborhoods in Toronto, we will keep 10 neighborhoods with the lowest crimes rates and again confront them to the remaining list of venues we've got.

Finally, we will add the prices of commercial housing in the actual list of venues and displays the more affordable ones.

Thanks for reading.

### Exploratory analysis

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

print("Libraries imported succesfully ! ")

Libraries imported succesfully ! 


In [48]:
def get_coordinates(address, verbose=False):
    try:
        geolocator = Nominatim(user_agent="tor_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude        
        lat = latitude
        lon = longitude
        return [lat, lon]
    except:
        return [None, None]

#address = 'Alexanderplatz, Berlin, Germany'
address = 'Toronto, Canada'
toronto_center = get_coordinates(address)
print('Coordinates of {}: {}'.format(address, toronto_center))

Coordinates of Toronto, Canada: [43.6534817, -79.3839347]


In [49]:
import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Toronto center longitude={}, latitude={}'.format(toronto_center[1], toronto_center[0]))
x, y = lonlat_to_xy(toronto_center[1], toronto_center[0])
print('Toronto center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Toronto center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Toronto center longitude=-79.3839347, latitude=43.6534817
Toronto center UTM X=-5310527.241020994, Y=10507538.454385541
Toronto center longitude=-79.3839347000005, latitude=43.65348169999977


In [132]:
k = (math.sqrt(3) / 2)*300
o = range(0, int(15/k))
o

range(0, 0)

In [174]:
toronto_center_x, toronto_center_y = lonlat_to_xy(toronto_center[1], toronto_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
#x_min = toronto_center_x - 6000
x_min = toronto_center_x - 3000
#x_step = 600
x_step = 400
#y_min = toronto_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_min = toronto_center_y - 4000 - (int(15/k)*k*400 - 8000)/2
#y_step = 600 * k 
y_step = 400 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
# for i in range(0, int(21/k)):
for i in range(0, int(15/k)):
    y = y_min + i * y_step
#    x_offset = 300 if i%2==0 else 0
    x_offset = 400 if i%2==0 else 0
#    for j in range(0, 21):
    for j in range(0, 15):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(toronto_center_x, toronto_center_y, x, y)
#        if (distance_from_center <= 6001):
        if (distance_from_center <= 4001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

254 candidate neighborhood centers generated.


In [133]:
# distances_from_center

In [8]:
import folium

In [184]:
map_toronto = folium.Map(location=toronto_center, zoom_start=13)
folium.Marker(toronto_center, popup='Toronto').add_to(map_toronto)
for lat, lon in zip(latitudes, longitudes):
    # folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    # folium.Circle([lat, lon], radius=232, color='blue', fill=False).add_to(map_toronto)
    # folium.Circle([lat, lon], radius=232, color='blue', fill=False).add_to(map_toronto)
    folium.features.RegularPolygonMarker([lat, lon], number_of_sides=4, rotation=40, radius=5, color='blue').add_to(map_toronto)
    # folium.Marker([lat, lon]).add_to(map_berlin)
map_toronto


#### Reverse geocoding

In [117]:

def get_address(latitude, longitude, verbose=False):
    try:
        
        locator = Nominatim(user_agent="myGeocoder")
        coordinates = str(latitude) + ", " + str(longitude)
#        coordinates = “53.480837, -2.244914”
        location = locator.reverse(coordinates)
        location.raw
        
        
        
#        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
#        response = requests.get(url).json()
#        if verbose:
#            print('Google Maps API JSON result =>', response)
#        results = response['results']
#        address = results[0]['formatted_address']
        #return address
        #return location.raw
        return location.address
    except:
        return None

addr = get_address(toronto_center[0], toronto_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(toronto_center[0], toronto_center[1], addr))

Reverse geocoding check
-----------------------
Address of [43.6534817, -79.3839347] is: None


In [176]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    #address = address.replace(', Canada', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [178]:
addresses

['NO ADDRESS',
 '2, Polson Street, Port Lands, Toronto—Danforth, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 1A4, Canada',
 'NO ADDRESS',
 'Silo, Martin Goodman Trail, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 3C4, Canada',
 '68, Distillery Lane, Distillery District, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 3C4, Canada',
 'Green P Carpark 289, 373, Front Street East, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 1H1, Canada',
 'NO ADDRESS',
 '68, Sumach Street, Corktown, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 3J7, Canada',
 'NO ADDRESS',
 '676, Dundas Street East, Regent Park, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 2B7, Canada',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'Bayview Cycle Path, Don Vale, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M4W 1J4, Canada',
 "Third Street #12 Sewage Pumping Station, 36, Bay

In [177]:
addresses_filled = []
list_size = len(addresses)
for cpt in range(0, list_size):
    if addresses[cpt] != 'NO ADDRESS' :
        addresses_filled.append(addresses[cpt])

print("Taille de liste: ", len(addresses_filled))
addresses_filled

Taille de liste:  93


['2, Polson Street, Port Lands, Toronto—Danforth, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 1A4, Canada',
 'Silo, Martin Goodman Trail, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 3C4, Canada',
 '68, Distillery Lane, Distillery District, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 3C4, Canada',
 'Green P Carpark 289, 373, Front Street East, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 1H1, Canada',
 '68, Sumach Street, Corktown, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 3J7, Canada',
 '676, Dundas Street East, Regent Park, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5A 2B7, Canada',
 'Bayview Cycle Path, Don Vale, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M4W 1J4, Canada',
 "Third Street #12 Sewage Pumping Station, 36, Bayview Avenue, Ward's Island, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5J 1Z4, 

We are going to concentrate our work on Downtown Toronto, as we were able to gather more informations on that specific part of the city.

First, we are going to put this data into a dataframe

In [183]:
stringsliced = addresses_filled[0].split(", ")
stringsliced

['2',
 'Polson Street',
 'Port Lands',
 'Toronto—Danforth',
 'Old Toronto',
 'Toronto',
 'Golden Horseshoe',
 'Ontario',
 'M5A 1A4',
 'Canada']

In [ ]:
df_addresses = pd.DataFrame('Address': addresses_filled,
                            ''
    )

In [8]:
tor_df = pd.read_csv('Canadian_postal_codes.csv')
tor_df.head()

,Unnamed: 0,Postal code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [17]:
tor_df_grouped = tor_df.groupby('Borough').count()
tor_df_grouped.head(20)

,Unnamed: 0,Postal code,Neighborhood
Borough,,,
Central Toronto,9,9,9
Downtown Toronto,19,19,19
East Toronto,5,5,5
East York,5,5,5
Etobicoke,12,12,12
Mississauga,1,1,1
North York,24,24,24
Scarborough,17,17,17
West Toronto,6,6,6


In [13]:
address = 'Downtown Toronto, Toronto, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [14]:
CLIENT_ID = '5BN44E0RZE5WZ2O3AEHZITCLUZEVWIQ1VW2SJF3YIEOZV0SV' # your Foursquare ID
CLIENT_SECRET = 'T0GYPQPSHMOES5J34VIYJVYF1VICCMWWDAEVZEKTPLPQ4F2W' # your Foursquare Secret
VERSION = '20200409' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5BN44E0RZE5WZ2O3AEHZITCLUZEVWIQ1VW2SJF3YIEOZV0SV
CLIENT_SECRET:T0GYPQPSHMOES5J34VIYJVYF1VICCMWWDAEVZEKTPLPQ4F2W


In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)


In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9dd9b495feaf001b5da881'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 233,
  'suggestedBounds': {'ne': {'lat': 43.67432211800001,
    'lng': -79.35608324781953},
   'sw': {'lat': 43.63832208199998, 'lng': -79.40574895218046}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57eda381498ebe0e6ef40972',
       'name': 'UNIQLO ユニクロ',
       'location': {'address': '220 Yonge St',
        'crossStreet': 'at Dundas St W',
        'lat': 43.65591027779457,
        'lng': -79.38064099181345,
        'labele

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
1,Ed Mirvish Theatre,Theater,43.655102,-79.379768
2,Yonge-Dundas Square,Plaza,43.656054,-79.380495
3,Hailed Coffee,Coffee Shop,43.658833,-79.383684
4,Banh Mi Boys,Sandwich Place,43.659292,-79.381949


In [62]:
food_category = '4d4b7105d754a06374d81259'

tropical_restaurant_category = ['4bf58dd8d48988d1c8941735','4bf58dd8d48988d145941735',
                              '4eb1d5724b900d56c88a45fe','4bf58dd8d48988d149941735',
                              '52e81612bcbc57f1066b7a03','4bf58dd8d48988d16b941735',
                              '4eb1bfa43b7b52c0e1adc2e8','58daa1558bbb0b01f18ec1f4',
                              '56aa371be4b08b9a8d573558','4bf58dd8d48988d1c1941735']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [63]:
import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    tropical_restaurants = {}
    location_restaurants = []
    foursquare_client_id = CLIENT_ID
    foursquare_client_secret = CLIENT_SECRET

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_tropical = is_restaurant(venue_categories, specific_filter=tropical_restaurant_category)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_tropical, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_tropical:
                    tropical_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, tropical_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
tropical_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('tropical_restaurants_350.pkl', 'rb') as f:
        tropical_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, tropical_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('tropical_restaurants_350.pkl', 'wb') as f:
        pickle.dump(tropical_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [64]:
print('Total number of restaurants:', len(restaurants))
print('Total number of Tropical restaurants:', len(tropical_restaurants))
print('Percentage of Tropical restaurants: {:.2f}%'.format(len(tropical_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 582
Total number of Tropical restaurants: 89
Percentage of Tropical restaurants: 15.29%
Average number of restaurants in neighborhood: 12.2991452991453


In [65]:
print('List of Tropical restaurants')
print('---------------------------')
for r in list(tropical_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(tropical_restaurants))

List of Tropical restaurants
---------------------------
('5542ab36498e2f92a8c248f2', 'Cocina Economica', 43.65495889022676, -79.3656572507398, '114 Berkeley St (btwn Queen & Richmond), Toronto ON, Canada', 327, True, -5310525.329727536, 10505400.979364943)
('58718c5b809a7743ecda3164', 'Mengrai Gourmet Thai', 43.654102, -79.3668, '82 Ontario St, Toronto ON M5A 2V3, Canada', 280, True, -5310647.127628458, 10505548.02396835)
('4bace084f964a520ca143be3', 'Oriental Taste', 43.655304, -79.365312, '329 Queen st (Parliment), Toronto ON, Canada', 349, True, -5310474.811746665, 10505355.043916743)
('4ada57aff964a520972121e3', 'Sukhothai', 43.658444465539525, -79.36568085121628, '274 Parliament St (btwn Coatsworth St. & Shuter St.), Toronto ON M5A 2Z6, Canada', 293, True, -5309970.437504293, 10505342.606025802)
('5532f36f498e09dc3233709d', 'Thai Room - Cabbagetown', 43.6641587260231, -79.36818872980979, '243 Carlton St, Toronto ON M5A 2L2, Canada', 243, True, -5309029.3970176065, 10505532.091795

In [66]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: Richmond Station, Cactus Club Cafe, The Chase, John & Sons Oyster House, John & Sons Oyster House, Leña, Hy's Steakhouse, Kupfert & Kim (First Canadian Place)
Restaurants around location 102: Richmond Station, The Chase, John & Sons Oyster House, John & Sons Oyster House, Leña, Noodle King, Cactus Club Cafe, Bannock
Restaurants around location 103: JOEY Eaton Centre, Richmond Station, Trattoria Mercatto, Bannock, The Senator Restaurant, Leña, JOEY, Paramount Fine Foods
Restaurants around location 104: JOEY Eaton Centre, The Senator Restaurant, Trattoria Mercatto, Salad King, Red Lobster, JOEY, Barberian's Steak House, Chipotle Mexican Grill
Restaurants around location 105: The Elm Tree Restaurant, Barberian's Steak House, Salad King, JOEY Eaton Centre, Scaddabush Italian Kitchen & Bar, Red Lobster, JOEY, Chipotle Mexican Grill
Restaurants around location 106: Barberian's Steak House, Salad King, Ki

In [67]:
map_toronto = folium.Map(location=toronto_center, zoom_start=13)
folium.Marker(toronto_center, popup='Toronto').add_to(map_toronto)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_tropical = res[6]
    color = 'red' if is_tropical else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_toronto)
map_toronto


In this map, red points are all tropical restaurants registered in Toronto.

According to this map, we can observe that tropical restaurants are not as widespread as someone could think in a city like Toronto. 
This also means that the search for a place for Noor's restaurant will be easier as there are  lot of locations where our restaurant could be settled.
